In [ ]:
import pandas as pd
import pickle

import sys
import os

sys.path.append(os.path.abspath(os.path.join('../src')))  # Adjust as needed

from distance_utils import distance
from data_load import load_tables
from filtering import filter_labors_by_date, filter_labors_by_city, filter_labores
from metrics import collect_vt_metrics_range, show_day_report_dayonly, compute_indicators
from preprocessing import remap_to_base_date, build_services_map_df, process_group
from plotting import plot_results
from config import *
from algorithms import remove_drivers, compute_avg_times, run_iteration_chronological, init_drivers

from multiprocessing import Pool

In [3]:

import time

def procesar_grupo_de_tres(chunk):
    """Función que procesa un grupo de 3 elementos"""
    print(f"Procesando grupo: {chunk}")
    # Simulamos algún trabajo pesado
    time.sleep(1)
    # Devolvemos la suma del grupo
    return sum(chunk)

def dividir_en_grupos(datos, tamaño_grupo=3):
    """Divide una lista en grupos del tamaño especificado"""
    grupos = []
    for i in range(0, len(datos), tamaño_grupo):
        grupo = datos[i:i + tamaño_grupo]
        grupos.append(grupo)
    return grupos

# if _name_ == "_main_":
# Datos de ejemplo
numeros = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

# Dividir en grupos de 3
grupos = dividir_en_grupos(numeros, 3)
print(f"Grupos creados: {grupos}")
# Resultado: [[1, 2, 3], [4, 5, 6], [7, 8, 9], [10, 11, 12], [13, 14, 15]]

# Procesamiento secuencial (para comparar)
print("\n--- Procesamiento Secuencial ---")
inicio = time.time()
resultados_secuencial = []
for grupo in grupos:
    resultado = procesar_grupo_de_tres(grupo)
    resultados_secuencial.append(resultado)
tiempo_secuencial = time.time() - inicio
print(f"Resultados secuenciales: {resultados_secuencial}")
print(f"Tiempo secuencial: {tiempo_secuencial:.2f}s")

# Procesamiento paralelo con pool.map
print("\n--- Procesamiento Paralelo ---")
inicio = time.time()
with Pool(processes=2) as pool:
    resultados_paralelo = pool.map(procesar_grupo_de_tres, grupos)
tiempo_paralelo = time.time() - inicio
print(f"Resultados paralelos: {resultados_paralelo}")
print(f"Tiempo paralelo: {tiempo_paralelo:.2f}s")

Grupos creados: [[1, 2, 3], [4, 5, 6], [7, 8, 9], [10, 11, 12], [13, 14, 15]]

--- Procesamiento Secuencial ---
Procesando grupo: [1, 2, 3]
Procesando grupo: [4, 5, 6]
Procesando grupo: [7, 8, 9]
Procesando grupo: [10, 11, 12]
Procesando grupo: [13, 14, 15]
Resultados secuenciales: [6, 15, 24, 33, 42]
Tiempo secuencial: 5.02s

--- Procesamiento Paralelo ---


Process SpawnPoolWorker-11:
Process SpawnPoolWorker-10:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/juanbeta/micromamba/envs/AlfredEnv/lib/python3.13/multiprocessing/process.py", line 313, in _bootstrap
    self.run()
    ~~~~~~~~^^
  File "/Users/juanbeta/micromamba/envs/AlfredEnv/lib/python3.13/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
    ~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/juanbeta/micromamba/envs/AlfredEnv/lib/python3.13/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/Users/juanbeta/micromamba/envs/AlfredEnv/lib/python3.13/multiprocessing/queues.py", line 387, in get
    return _ForkingPickler.loads(res)
           ~~~~~~~~~~~~~~~~~~~~~^^^^^
AttributeError: Can't get attribute 'procesar_grupo_de_tres' on <module '__main__' (<class '_frozen_importlib.BuiltinImporter'>)>
  File "/Users/juanbeta/micromamba/envs/AlfredEnv/lib/python3.13/multiprocessin

KeyboardInterrupt: 

In [ ]:
import pandas as pd
import pickle
from datetime import timedelta
from multiprocessing import Pool

# ——————————————————————————
# Función maestra por ciudad
# ——————————————————————————
def run_city_pipeline(city_code, start_date):
    """
    Ejecuta TODO el flujo para una ciudad y fecha dada.
    start_date: string o Timestamp, mismo para todas las ciudades.
    Devuelve: (city_code, df_cleaned, df_moves)
    """
    # 1. Filtrar por ciudad
    df_city = filter_labors_by_city(df_dist, str(city_code))
    
    # 2. Filtrar por rango de fechas (start_date -> start_date+1)
    end_date = pd.to_datetime(start_date) + timedelta(days=1)
    df_day = filter_labors_by_date(df_city, start_date=start_date, end_date=end_date)
    
    # 3. Quitar cancelados y ordenar
    df_day = (
        df_day.query("state_service != 'CANCELED'")
        .sort_values(['service_id', 'labor_start_date'])
        .reset_index(drop=True)
    )

    # 4. Remapear fechas al día base
    base_day = pd.to_datetime(start_date).date()
    df_day = remap_to_base_date(
        df_day, 
        ['schedule_date', 'labor_start_date', 'labor_end_date'], 
        base_day
    )

    # 5. Construir mapa de servicios
    services_map_df = build_services_map_df(df_day)

    # 6. Procesar grupos
    cleaned = [
        process_group(grp, DISTANCE_METHOD, DIST_DICT) 
        for _, grp in df_day.groupby('service_id', sort=False)
    ]
    df_cleaned_template = pd.concat([c for c in cleaned if not c.empty], ignore_index=True)
    df_cleaned_template = df_cleaned_template.merge(
        services_map_df, 
        on=['service_id', 'labor_id'], 
        how='left'
    )

    # 7. Ejecutar la iteración cronológica
    df_cleaned, df_moves = run_iteration_chronological(df_cleaned_template)

    # 8. Devolver resultados
    return city_code, df_cleaned, df_moves


# ——————————————————————————
# Configuración previa
# ——————————————————————————
if __name__ == "__main__":
    # Pre-cargar globals para todos
    with open('distances.pkl', 'rb') as f:
        DIST_DICT = pickle.load(f)

    df_dist = pd.read_pickle("df_dist.pkl")

    # Lista de ciudades
    city_list = cities_df['cod_ciudad'].tolist()

    # Parámetros de fecha
    start_date = "2023-01-02"

    # Paralelizar
    with Pool(processes=4) as pool:  # ajusta procesos según tu CPU
        results = pool.starmap(run_city_pipeline, [(c, start_date) for c in city_list])

    # Resultados como diccionario
    results_by_city = {city: (df_cleaned, df_moves) for city, df_cleaned, df_moves in results}
